Import the data provided in the `all_states.json` file from your Terminal. Name the database `birth_data` and the collection `states`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d birth_data -c states --drop --jsonArray all_states.json`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from api_key import api_key

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
print(mongo.list_database_names())

In [ ]:
# assign the uk_food database to a variable name
db = mongo["birth_data"]

In [ ]:
# review the collections in our new database
print(db.list_collection_names())

In [ ]:
# review a document in the establishments collection
pprint(db.states.find_one())

In [ ]:
# assign the collection to a variable
states = db['states']

Add in Coordinates for each players' birth location

Using Geoapify

In [ ]:
# Adding Lat & Lon to the DataBase
db.states.update_many(
    {},
    {
        '$set': {
            'Lat': '',  
            'Lon': '' 
        }
    },
    upsert=True
)

In [ ]:
# review a document in the establishments collection
pprint(db.states.find_one())

In [ ]:
import requests

filter = {} 

# Iterate through documents in the collection
for document in states.find(filter):
    # Extract "City" and "State" values from the document
    city = document['City'].replace(" ", "_")
    state = document['State']
    
    # Url for API request
    target_url = f"https://api.geoapify.com/v1/geocode/search?city={city}&state={state}&format=json&apiKey={api_key}"

    # Make API request to get latitude and longitude
    response = requests.get(target_url)
    data = response.json()

    # Extract latitude and longitude from the API response
    if 'results' in data and data['results']:
        lat = data['results'][0]['lat']
        lon = data['results'][0]['lon']
        print(f'Coordinates for {city},{state}: lat: {lat}, lon: {lon}')

        # Update the document in the collection with latitude and longitude
        states.update_one({'_id': document['_id']}, {'$set': {'Lat': lat, 'Lon': lon}})

In [ ]:
# review a document in the establishments collection
pprint(db.states.find_one())